In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from tqdm import tqdm
import seaborn as sns
import re
import datetime
import os
import random
import h3 
import shap 

c:\Users\Yuan\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
h3_l7_df = pd.read_csv('h3_l7_df_new.csv')

In [4]:
# 設定plt環境
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [5]:
train_index=[]
test_index=[]
for i in range(0,h3_l7_df.shape[0]):
    geo_location = h3.h3_to_geo(h3_l7_df.iloc[i]['id'])

    if (geo_location[1]) > (-76.05): #把經度大於-76.05的 當train (東邊是train)
        train_index.append(i)
    else:
        test_index.append(i)

# 分割訓練集和測試集
train_h3_l7_df = h3_l7_df.iloc[train_index]
test_h3_l7_df = h3_l7_df.iloc[test_index]


In [6]:
# 將 h3_l7_df 資料框中的 'id' 列移除，僅保留數據進行正規化
h3_spatial_data = h3_l7_df.drop('id', axis=1)


# # # 對數據進行正規化：將每個數據列的最小值調整為 0，最大值調整為 1
normalized_spatial_data = (h3_spatial_data - h3_spatial_data.min()) / (h3_spatial_data.max() - h3_spatial_data.min())

#設定OHCA正規化反函數 方便把預測結果返回原本scale
ohca_reguli_inverse = (h3_l7_df.ohca.max()-h3_l7_df.ohca.min()) + h3_l7_df.ohca.min()

# 將 DataFrame 轉換為 numpy array，並設定數據類型為 np.float64
spatial_data = np.array(normalized_spatial_data).astype(np.float64)


train_spatial_data = spatial_data[train_index]
test_spatial_data = spatial_data[test_index]

print(len(train_index))
print(len(test_index))

83
94


In [7]:
class Regressor(nn.Module):
    """
    用於迴歸任務的神經網絡模型 Regressor。
    
    結構:
    - 兩層隱藏層，並使用 ReLU 激活函數
    - 最後一層為線性層，不使用激活函數（適用於迴歸）
    """
    def __init__(self, input_size=2, hidden_size=32, output_size=1):
        super().__init__()
        # 定義三層全連接層
        self.fc1 = nn.Linear(input_size, hidden_size)     # 第一層：輸入層到隱藏層
        self.fc2 = nn.Linear(hidden_size, hidden_size)    # 第二層：隱藏層到隱藏層
        self.fc3 = nn.Linear(hidden_size, output_size)    # 第三層：隱藏層到輸出層

        # 初始化權重和偏置
        nn.init.normal_(self.fc1.weight, std=0.02)
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.normal_(self.fc2.weight, std=0.02)
        nn.init.constant_(self.fc2.bias, 0)
        nn.init.normal_(self.fc3.weight, std=0.02)
        nn.init.constant_(self.fc3.bias, 0)
        
    def forward(self, input):
        # 前向傳播過程
        output = F.relu(self.fc1(input))  # 第一層 + ReLU 激活
        output = F.relu(self.fc2(output)) # 第二層 + ReLU 激活
        output = self.fc3(output)         # 第三層（不使用激活函數）
        return output

In [8]:
window_size = 1
seed = 7890 #7890
torch.manual_seed(seed)
np.random.seed(seed)

def train_reg(spatial_data, 
              s_net,
              s_net_optim, 
              window_size, iter_num=5000):
    """
    訓練 s_net  網絡來預測 spatial_data 中的數據。
    
    參數:
    - spatial_data: numpy array，包含訓練數據
    - s_net: 神經網絡模型
    - s_net_optim: 優化器
    - window_size: 每次迭代的隨機取樣大小
    - iter_num: 訓練迭代次數
    
    返回:
    - loss_array: 每次迭代的損失值
    - t_fea_array, s_fea_array: 用於存儲特徵的暫時性陣列（目前未使用）
    """

    loss_array = []     # 儲存每次迭代的損失
    t_fea_array = []    # 預留用於儲存暫時性特徵的空列表
    s_fea_array = []    # 預留用於儲存暫時性特徵的空列表

    for _ in tqdm(range(iter_num)):
        
        # 隨機選擇一組數據索引
        h3_l7_id = np.random.choice(spatial_data.shape[0] - 1, window_size)


        # 提取目標變數（即輸入的最後一列數據）並轉為 Tensor
        ohca = spatial_data[h3_l7_id, -1].reshape(-1, 1)
        ohca = torch.autograd.Variable(torch.FloatTensor(ohca))

        # p_pred 用於預測目標變數
        p_pred = s_net(torch.autograd.Variable(torch.FloatTensor(spatial_data[h3_l7_id, :-1]))).reshape(-1, 1)

        # 定義均方誤差損失
        mseloss = torch.nn.MSELoss(reduction='sum')
        loss = mseloss(p_pred, ohca)
        
        # 清空前一次計算的梯度
        s_net_optim.zero_grad()
        
        
        # 計算損失的梯度
        autograd.backward(loss)

        # 更新神經網絡參數
        s_net_optim.step()
        
        # 儲存損失值
        loss_array.append(loss.detach().cpu().numpy())

    return loss_array, t_fea_array, s_fea_array

# 初始化模型和優化器
s_net = Regressor(input_size=spatial_data.shape[1] - 1, hidden_size=spatial_data.shape[1] * 2, output_size=1)
s_net_optim = optim.Adam(s_net.parameters(), lr=1e-3, weight_decay=1e-5)

iter_num=30000
# 執行訓練過程
loss_array, t_fea_array, s_fea_array = train_reg(train_spatial_data, s_net,
                                                 s_net_optim,
                                                 window_size, iter_num)

100%|██████████| 30000/30000 [01:03<00:00, 469.84it/s]


In [9]:
y_head_train = s_net(torch.autograd.Variable(torch.FloatTensor(train_spatial_data[:, :-1]))).detach().numpy()*ohca_reguli_inverse
y_train = train_spatial_data[:, -1]*ohca_reguli_inverse
y_head_test = s_net(torch.autograd.Variable(torch.FloatTensor(test_spatial_data[:, :-1]))).detach().numpy()*ohca_reguli_inverse
y_test = test_spatial_data[:, -1].reshape(-1, 1)*ohca_reguli_inverse

In [10]:
total_sum = np.sum(y_test)
total_sum

1504.0

In [11]:
mae = np.abs(y_head_test-y_test)
ans_mae = mae.sum()/mae.shape[0]

print('MAE of test set= ',ans_mae)

# 計算殘差變異
ss_residual = np.sum((y_test - y_head_test) ** 2)

# 計算總變異量
ss_total = np.sum((y_test - np.mean(y_test)) ** 2)

# 計算 R²
r_squared = 1 - (ss_residual / ss_total)

n = mae.shape[0]          # Number of data points
p = train_spatial_data.shape[1]            # Number of predictors

# Adjusted R-squared calculation
r_squared_adj = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)

print("R² of test set= ", r_squared)
print("ADJ R² of test set= ", r_squared_adj)

MAE of test set=  6.406070350332463
R² of test set=  0.6517364254708865
ADJ R² of test set=  2.408196192661198


# SHAP

In [12]:
# 假設 `spatial_data` 包含背景數據，用於 SHAP 的解釋
background_data = torch.FloatTensor(train_spatial_data[:, :-1])  
test_data = torch.FloatTensor(test_spatial_data[:, :-1])  

In [13]:
# 建立 SHAP 解釋器，使用背景數據
explainer = shap.GradientExplainer(s_net, background_data)
shap_values_test = explainer.shap_values(test_data)*ohca_reguli_inverse
# Get the shap values from my test data

test_features_df = h3_spatial_data.iloc[:, :-1]
feature_names = test_features_df.columns

KeyboardInterrupt: 

In [144]:
#把SHAP 換成壞圖之格式
shap_col = shap_values_test.shape[0]
shap_row = shap_values_test.shape[1]
shap_values_test_2D = shap_values_test.reshape(shap_col,shap_row)
# shap.summary_plot(shap_values_test_2D, test_data,feature_names)

In [145]:
feature_names_w_SHAP = [f'shap {col}' for col in feature_names] # 在每個列名前加上 'shap'
SHAP_df = pd.DataFrame(shap_values_test_2D, columns=feature_names_w_SHAP) #換成 DF

In [146]:
df1 = test_h3_l7_df.reset_index(drop=True)
df2 = SHAP_df.reset_index(drop=True)
test_h3_l7_df_S = pd.concat([df1, df2], axis=1) #合併SHAP值到test_h3_l7_df

In [ ]:
# 初始化結果 DataFrame
spatial_data_score = pd.DataFrame()
spatial_data_score['id'] = test_h3_l7_df_S['id']

# 初始化字典來存儲計算結果
results_dict = {}
# 逐列處理
for col in feature_names:
    col_result = []  # 存儲當前特徵的計算結果
    
    # 遍歷每一行
    for row in range(test_h3_l7_df_S.shape[0]):
        building_name = col
        shap_name = 'shap ' + building_name
        denominator = test_h3_l7_df_S.iloc[row][building_name] # 分母
        numerator = test_h3_l7_df_S.iloc[row][shap_name]  # 分子
        
        # 處理分母為 0 的情況
        if denominator == 0:
            col_result.append(numerator)
        else:
            col_result.append(numerator / denominator)

    # 將結果存入字典
    results_dict[col] = col_result

# 一次性加入所有計算結果
spatial_data_score = pd.concat([spatial_data_score, pd.DataFrame(results_dict)], axis=1)

In [148]:
# # 初始化結果 DataFrame
# spatial_data_score = pd.DataFrame()
# spatial_data_score['id'] = test_h3_l7_df_S['id']

# # 循環處理每一列
# for col in range(feature_names.shape[0]):
#     col_result = []  # 用於存儲當前列的計算結果
    
#     # 遍歷每一行
#     for row in range(test_h3_l7_df_S.shape[0]):
#         denominator = test_h3_l7_df_S.iloc[row, col + 1]  # 分母
#         numerator = test_h3_l7_df_S.iloc[row, col + feature_names.shape[0] + 3]  # 分子
        
#         # 如果分母為 0，直接使用原分子數據
#         if denominator == 0:
#             col_result.append(numerator)
#         else:
#             col_result.append(numerator / denominator)  # 正常執行除法
    
#     # 將當前列結果存入結果 DataFrame
#     spatial_data_score[feature_names[col]] = col_result


# 製造有SHAP值的poi_df

In [149]:
poi_df = pd.read_csv('poi_df.csv') #讀進原檔案

In [150]:
selected_values = spatial_data_score['id']
test_poi_df = poi_df[poi_df['h3_l7'].isin(selected_values)]

In [151]:

test_poi_df = test_poi_df.reset_index(drop=True)
test_poi_df['score'] = None  # 初始化 'score' 列為空值

# 循環處理每一行
for i in range(0, test_poi_df.shape[0]):
    poi_id = test_poi_df['h3_l7'].iloc[i]  # 取得當前行的 poi_id
    building_type = test_poi_df['building'].iloc[i]  # 取得當前行的 building_type
    
    # 查找 spatial_data_score 中對應的 id 和 amenity
    positions = spatial_data_score.index[spatial_data_score['id'] == poi_id]
 
        # 檢查 building_type 是否在 spatial_data_score 的列中
    if building_type in spatial_data_score.columns:
        building_score = spatial_data_score.loc[positions, building_type]
        
        if not building_score.empty:
            # 如果找到了對應的建築分數，將其轉換為數字並儲存
            test_poi_df.loc[i, 'score'] = pd.to_numeric(building_score.iloc[0])
        else:
            # 如果沒有找到對應的 building_type，可以設為 NaN 或其他預設值
            test_poi_df.loc[i, 'score'] = None
    else:
        # 如果 building_type 不存在於 spatial_data_score，設為 NaN 或其他預設值
        test_poi_df.loc[i, 'score'] = None


# 檢查結果
test_poi_df

,osmid,lat,lon,h3_l7,building,score
0,356567877,36.709595,-76.047987,872af626dffffff,place_of_worship,0.086722
1,356567948,36.586005,-76.082863,872af051cffffff,place_of_worship,0.373893
2,356568033,36.733205,-76.097433,872af626effffff,place_of_worship,0.357984
3,356568069,36.557653,-76.073542,872af0519ffffff,place_of_worship,0.110818
4,356568361,36.850422,-76.160777,872af6353ffffff,place_of_worship,-0.167937
...,...,...,...,...,...,...
99719,7485109,36.866139,-76.133943,872af6350ffffff,residential,-0.0187
99720,9637418,36.858806,-76.170325,872af622cffffff,house,0.001472
99721,11674759,36.819752,-76.076312,872af634effffff,school,0.595646
99722,12032839,36.790276,-76.111886,872af6266ffffff,residential,-0.009574


In [152]:
test_poi_df['score']

0        0.086722
1        0.373893
2        0.357984
3        0.110818
4       -0.167937
           ...   
99719     -0.0187
99720    0.001472
99721    0.595646
99722   -0.009574
99723    0.048088
Name: score, Length: 99724, dtype: object

In [153]:
sum(list(test_poi_df['score']))

803.4365794421532

In [154]:
test_poi_df.to_csv('test_poi_df.csv', index=False, sep=',', encoding='utf-8-sig')

In [155]:
def intersection_area(r1, r2, d):
    if d >= r1 + r2:
        return 0  # 兩圓不相交
    elif d <= abs(r1 - r2):
        return np.pi * min(r1, r2)**2  # 一圓包含另一圓
    else:
        # 使用公式計算相交面積
        term1 = r1**2 * np.arccos((d**2 + r1**2 - r2**2) / (2 * d * r1))
        term2 = r2**2 * np.arccos((d**2 + r2**2 - r1**2) / (2 * d * r2))
        term3 = 0.5 * np.sqrt((-d + r1 + r2) * (d + r1 - r2) * (d - r1 + r2) * (d + r1 + r2))
        return term1 + term2 - term3


In [156]:
import math

#緯度（latitude） 經度（longitude）
def haversine(lat1, lon1, lat2, lon2):
    # 將經緯度轉換為弧度
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # 計算差異
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # 哈弗辛公式
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # 地球半徑（公里）
    R = 6371.0
    
    # 計算距離
    distance = R * c #單位為km
    return distance 


In [165]:
a = test_poi_df[test_poi_df['h3_l7'] == '872af6353ffffff']
sum(a.score)

16.839208192292585

In [158]:
def area_score(AED_location, AED_range):
    score_list = []
    center_radius = 1.21  # 根據 H3 Level 7 的網格大小進行調整
    for i in range(0, test_poi_df.shape[0]):
        distance = haversine(AED_location[0], AED_location[1], test_poi_df.iloc[i]['lat'], test_poi_df.iloc[i]['lon'])
        
        # 如果 POI 在 AED 範圍外，跳過該 POI
        if distance > AED_range or test_poi_df.iloc[i]['score'] ==None:
            continue
        else:
            # 將 H3 網格編碼轉換為經緯度
            L7_center = h3.h3_to_geo(test_poi_df.iloc[i]['h3_l7'])
            center_distance = haversine(AED_location[0], AED_location[1], L7_center[0], L7_center[1])
            
            # 計算重疊面積的比例
            intersection_area_value = intersection_area(AED_range, center_radius, center_distance)
            proportion = intersection_area_value / ((center_radius ** 2) * np.pi)
            # 根據比例計算加權分數
            Score = test_poi_df.iloc[i]['score'] * proportion
            score_list.append(Score)
    
    return sum(score_list)  # 返回分數列表，方便後續處理


In [159]:
area_score(h3.h3_to_geo('872af6309ffffff'),0.9)

4.346658917662801

In [160]:
h3.h3_to_geo('872af6309ffffff')

(36.90609575498717, -76.14199701723689)

In [161]:
# ohca_df = pd.read_csv('OHCAs.csv')
# h3_l7 = []

# for i in range(ohca_df.shape[0]):
#     h3_l7.append(h3.geo_to_h3(ohca_df.Latitude[i], ohca_df.Longitude[i], resolution=7))

# ohca_df['h3_l7'] = h3_l7

# if min_lat < min(ohca_df['Latitude']): min_lat = min(ohca_df['Latitude'])
# if max_lat > max(ohca_df['Latitude']): max_lat = max(ohca_df['Latitude'])
# if min_lon < min(ohca_df['Longitude']): min_lon = min(ohca_df['Longitude'])
# if max_lon > max(ohca_df['Longitude']): max_lon = max(ohca_df['Longitude'])

# ohca_df = ohca_df.drop_duplicates(subset=['ReceivedTime', 'Latitude', 'Longitude'])
# ohca_df['ReceivedTime'] = pd.to_datetime(ohca_df['ReceivedTime'])
# ohca_df['ReceivedTime'] = ohca_df['ReceivedTime'].apply(lambda x: x.date())

In [162]:
# cols = np.concatenate((
#             poi_df.amenity.unique(),
#         ))
# len(cols)

# h3_l7_df = pd.DataFrame(data={'id': np.unique(np.concatenate((poi_df.h3_l7.unique(), ohca_df.h3_l7.unique())))})
# h3_l7_df[cols] = 0

# for i in range(poi_df.shape[0]):
#     h3_l7_id = poi_df.iloc[i]['h3_l7']
#     h3_l7_df.loc[h3_l7_df['id'] == h3_l7_id, poi_df.iloc[i]['amenity']] += 1
